# Scala Test 4

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD

Intitializing Scala interpreter ...

Spark Web UI available at http://Laptop:4040
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1722326488412)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.sql.functions._
import org.apache.spark.rdd.RDD


In [2]:
val spark = SparkSession.builder()
                        .appName("Assignment 25072024")
                        .master("local[*]")
                        .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@3b6b0448


In [3]:
val sc = spark.sparkContext

sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@16f23455


In [2]:
import scala.math.sqrt
import scala.math.ceil

def isPrime(n: Int): Boolean = {
    if (n <= 1) {
        false
    } else {
        val upperBound = ceil(sqrt(n)).toInt
        !(2 to upperBound).exists(i => n % i == 0)
    }
}

import scala.math.sqrt
import scala.math.ceil
isPrime: (n: Int)Boolean


In [3]:
val numbers_file = "input_data/numbers.txt"
val prime_numbers_rdd_output_path = "output_data/prime_numbers_rdd.txt"

numbers_file: String = input_data/numbers.txt
prime_numbers_rdd_output_path: String = output_data/prime_numbers_rdd.txt


In [4]:
val numbers_rdd: RDD[String] = sc.textFile(numbers_file)

numbers_rdd: org.apache.spark.rdd.RDD[String] = input_data/numbers.txt MapPartitionsRDD[1] at textFile at <console>:33


In [6]:
val numbers_rdd_int: RDD[Int] = numbers_rdd.map(x => x.toInt)

numbers_rdd_int: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[2] at map at <console>:32


In [7]:
val primeNumbersRDD: RDD[Int] = numbers_rdd_int.filter(isPrime)

primeNumbersRDD: org.apache.spark.rdd.RDD[Int] = MapPartitionsRDD[3] at filter at <console>:33


# Exercise 2

In [10]:
val textFile_path = "input_data/textLine.txt"

textFile_path: String = input_data/textLine.txt


In [11]:
val rdd = sc.textFile(textFile_path)

rdd: org.apache.spark.rdd.RDD[String] = input_data/textLine.txt MapPartitionsRDD[5] at textFile at <console>:33


In [14]:
def convertToKeyValue(value: String): (Char, String) = {
    val parts = value.split(',').map(_.trim)
    val key = parts(0).head
    val valuePart = parts(1)
    (key, valuePart)
}


def convertToKeyLenValue(value: String): (Int, Int) = {
    val parts = value.split(',').map(_.trim)
    val key = parts(0).toInt
    val length = parts(1).length
    (key, length)
}


convertToKeyValue: (value: String)(Char, String)
convertToKeyLenValue: (value: String)(Int, Int)


In [16]:
val key_lenValue_rdd = rdd.map(convertToKeyLenValue)

key_lenValue_rdd: org.apache.spark.rdd.RDD[(Int, Int)] = MapPartitionsRDD[7] at map at <console>:33


In [17]:
key_lenValue_rdd.take(5)

res2: Array[(Int, Int)] = Array((49,21), (10,21), (39,21), (86,21), (69,21))


In [18]:
def sumCount(value: Iterable[Int]): (Int, Int) = {
    val totalSum = value.sum
    val count = value.size
    (totalSum, count)
}

def avgCalGroupByKey(value: (Int, Int)): Double = {
    val (sum, count) = value
    sum.toDouble / count
}


sumCount: (value: Iterable[Int])(Int, Int)
avgCalGroupByKey: (value: (Int, Int))Double


In [20]:
val grouped_rdd = key_lenValue_rdd.groupByKey()

val sum_count_rdd = grouped_rdd.mapValues(sumCount)

val avg_rdd_groupByKey = sum_count_rdd.mapValues(avgCalGroupByKey)

grouped_rdd: org.apache.spark.rdd.RDD[(Int, Iterable[Int])] = ShuffledRDD[11] at groupByKey at <console>:36
sum_count_rdd: org.apache.spark.rdd.RDD[(Int, (Int, Int))] = MapPartitionsRDD[12] at mapValues at <console>:38
avg_rdd_groupByKey: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[13] at mapValues at <console>:40


In [23]:
avg_rdd_groupByKey.collect()

res6: Array[(Int, Double)] = Array((96,23.92222222222222), (34,23.956989247311828), (52,23.89516129032258), (4,23.893617021276597), (16,23.892857142857142), (82,23.852631578947367), (66,23.916666666666668), (80,23.846153846153847), (54,23.9375), (28,23.894736842105264), (98,23.882978723404257), (30,23.90625), (50,23.855555555555554), (14,23.870967741935484), (92,23.840425531914892), (24,23.94736842105263), (64,23.921348314606742), (36,23.887755102040817), (74,23.877777777777776), (90,23.858585858585858), (72,23.858823529411765), (70,23.905982905982906), (18,23.8018018018018), (12,23.91919191919192), (38,23.876543209876544), (20,23.918181818181818), (78,23.945652173913043), (10,23.86868686868687), (94,23.914893617021278), (100,23.87719298245614), (84,23.875), (56,23.86021505376344), (76,...


In [24]:
def valueAndCount(value: Int): (Int, Int) = {
    (value, 1)
}

def sumLenValue(value1: (Int, Int), value2: (Int, Int)): (Int, Int) = {
    (value1._1 + value2._1, value1._2 + value2._2)
}

def avgCalReduceByKey(value: (Int, Int)): Double = {
    value._1.toDouble / value._2
}


valueAndCount: (value: Int)(Int, Int)
sumLenValue: (value1: (Int, Int), value2: (Int, Int))(Int, Int)
avgCalReduceByKey: (value: (Int, Int))Double


In [27]:
val sum_count_rdd = key_lenValue_rdd.mapValues(valueAndCount)
                                .reduceByKey(sumLenValue)

val avg_rdd_reduceByKey = sum_count_rdd.mapValues(avgCalReduceByKey)

sum_count_rdd: org.apache.spark.rdd.RDD[(Int, (Int, Int))] = ShuffledRDD[15] at reduceByKey at <console>:37
avg_rdd_reduceByKey: org.apache.spark.rdd.RDD[(Int, Double)] = MapPartitionsRDD[16] at mapValues at <console>:39


In [28]:
avg_rdd_reduceByKey.collect()

res7: Array[(Int, Double)] = Array((34,23.956989247311828), (52,23.89516129032258), (96,23.92222222222222), (4,23.893617021276597), (16,23.892857142857142), (82,23.852631578947367), (66,23.916666666666668), (28,23.894736842105264), (54,23.9375), (80,23.846153846153847), (98,23.882978723404257), (30,23.90625), (14,23.870967741935484), (50,23.855555555555554), (36,23.887755102040817), (24,23.94736842105263), (64,23.921348314606742), (92,23.840425531914892), (74,23.877777777777776), (90,23.858585858585858), (72,23.858823529411765), (70,23.905982905982906), (18,23.8018018018018), (12,23.91919191919192), (38,23.876543209876544), (20,23.918181818181818), (78,23.945652173913043), (10,23.86868686868687), (94,23.914893617021278), (84,23.875), (100,23.87719298245614), (56,23.86021505376344), (76,...
